# SWIFT notebook

You can find very detailed SWIFT documentation from this [link](http://swift.dur.ac.uk/docs/index.html). This notebook is only a very basic guide and should not be fully trusted. The choices of modules, compiler, MPI library and parameters is based on my previous test on BlueCrystal 4, BluePebble 1 and Trappist.

## <a name="top"></a>Contents:
[1. Compiling SWIFT](#S1) <br>
 * [Modules](#S11)<br>
 * [Configuration](#S12)<br>
 * [Compilation](#S13)<br>
 * [SLURM scripts](#S14)<br>
 * [Command line options](#S15)<br>

[2. Parameter file](#S2)

[3. Generating IC file](#S3)

[4. Read and Analysis](#S4)

---

## <a name="S1"></a>**1. Compiling SWIFT**

From [SWIFT gitlab](https://gitlab.cosma.dur.ac.uk/swift/swiftsim/-/tree/master) you can download the lastest version of SWIFT, it may fail to compile sometimes. If that's the case, try to download it from their [github](https://github.com/SWIFTSIM/swiftsim) which would be more stable. However, the lastest version means more fancy features, e.g. report of cpu dead time.

After downloading SWIFT, using following command to have an intial set up:
```bash
module purge
module load libtool

./autogen.sh
```

First things first, choose a compiler and an MPI library. Based on the architechtures of the HPC you can choose to use either gcc or intel icc. However, if the HPC has an OmniPath, Open MPI v3.1.3 or higher is suggested to be used. A bug in the `psm2` library causes communications to be lost. It is possible to run SWIFT with older versions v2.1.x of Open MPI so long as `psm` is used instead of `psm2`, i.e. that you invoke mpirun with `--mca btl vader,self -mca mtl psm` (Only do this if normal mpirun is not working)

Below are the necessary modules needed to utilize the most basic functions of SWIFT:
* HDF5 (SWIFT makes effective use of parallel HDF5 when running on more than one node, if possible, use a parallel version of HDF5 lib)
* GSL
* FFTW (SWIFT does not make use of the parallel capability of FFTW. Calculations are done by single MPI nodes independently.)

---

SWIFT can actually compile and run well with the above modules loaded, below are some bonus modules that can further optimize the code and the load balance on muti-core nodes. 
* ParMETIS/METIS (configure option: `--with-parmetis=/path/to/your/locally/installed/parmetis/lib`)
    * BC4 and BP1 have ParMETIS installed but SWIFT can't link to it sometimes, one option is to install the ParMETIS lib locally on your home dir. Download from this [link](http://glaros.dtc.umn.edu/gkhome/metis/parmetis/download) and follow the install instruction. You should first go to the dir `/path/to/lib/parmetis-4.0.3/metis/include/` and edit the file `metis.h`. In the file, change the line 33 from `#define IDXTYPEWIDTH 32` to `#define IDXTYPEWIDTH 64`. After that, at the top of ParMetis' directory execute `make` and follow the instructions. Make sure you provide a installation prefix by using `make config prefix=/some/dir/for/parmetis` . After installation of ParMETIS, please also install METIS using the same installation prefix. Direct to `metis` dir and at the top of Metis' directory execute `make` and follow the instructions. 
* LibNUMA
  * a build of the NUMA library can be used to pin the threads to the physical core of the machine SWIFT is running on. Normally no need to load  any modules on BC4 and PB1, but please use configure option:`--enable-compiler-warnings=yes` during  configuration step, otherwise, it will fail to compile(on bc4 and pb1).
* tcmalloc / jemalloc / TBBmalloc: (configure option:`--with-tcmalloc`, `--with-jemalloc` , `--with-tbbmalloc`)
  * a build of the tcmalloc library, jemalloc or TBBmalloc can be used to obtain faster and more scalable allocations than the standard C malloc function part of glibc. Using one of these is highly recommended on systems with many cores per node. Right now unavaiable on bc4, available on bp1. 

---

### <a name="S11"></a>**Modules**

Below are the modules recipe I currently use on the BC4:
```bash
module purge
module load apps/parmetis/4.0.3 
module load libs/hdf5/1.10.1.MPI 
module load libs/fftw/3.3.6 
module load libs/gsl/1.16-gcc-9.1.0 
module load languages/intel/2020-u4

module save swift_module
```

### <a name="S12"></a>**Configuration**

For the planetary simulation, you could use the following configuration options:
```
./configure --with-hydro=planetary \
            --with-equation-of-state=planetary \
            --enable-compiler-warnings=yes \
            --with-gravity=basic \
            --with-parmetis=/mnt/storage/home/YourAccount/PathTo/parmetis \
            CC=icc \ 
            MPICC=mpiicc

```

* SWIFT default compiler is gcc but you can use `CC` environment variable to select the prefered compiler. `./configure CC=icc` to change to use Intel compiler.
* The MPI compiler can be controlled using the `MPICC` variable, much like the `CC` one. 
* copy file `metis.h` from the `/dir/you/intall/parmetis/include` to `/dir/you/intall/swift/swiftsim/src/` would sometimes avoid the compilation error related to ParMETIS library. 
* You will need another SWIFT executable to run artificial cooling (velocity dumping is not included in the SWIFT). During configuration, add this option`--enable-planetary-fixed-entropy` will override the internal energy of each particle each step such that its specific entropy remains constant. Your initial conditions file must includes specific entropies for each particle in order to use this feature.
* add `--with-tcmalloc` to the above configure command if it's available on your HPC.
* `./configure --help` is pretty helpful, you can find all available options by using that. 

### <a name="S13"></a>**Compilation**

After configuration, if everything works fine, your will get a configuartion summary list which you can use to have a double check of the libraries and features you used. \
You can then just use `make` to compile the code. If things go well, you will find two executble `swift` and `swift_mpi` in the `/swiftsim/example` directory. Run `swift` if you are only using one node. Use `swift_mpi` with one MPI rank per NUMA region (per socket) for muti-node job.

---

### <a name="S14"></a>**SLURM scripts** 

Below are some SLURM scripts used on BC4 where each node has two chips with each chip has 14 CPUs(so in total 28 CPUs per node).

#### **<font color='green'>Single-node job with <font color='red'>all</font> cpus</font>**

```
#!/bin/bash -l

#SBATCH -J swift_impact_1node28cpu 
#SBATCH --nodes 1
#SBATCH --exclusive

module purge 
module restore swift_module

./swift -a -s -G -t 28 parameters_impact.yml

```

 #### **<font color='green'>Single-node job with <font color='red'>partial</font> cpus</font>**

```
#!/bin/bash -l

#SBATCH -J swift_impact_1node20cpu 
#SBATCH --nodes 1
#SBATCH --cpus-per-task=20        
#SBATCH --tasks-per-node=1

module purge
module restore swift_module

./swift -a -s -G -t 20 parameters_impact.yml
```

#### **<font color='green'>Multi-node job with <font color='red'>all</font> cpus</font>**

```
#!/bin/bash -l

#SBATCH -J swift_impact_2node56cpu
#SBATCH --tasks-per-node=2
#SBATCH --nodes 2
#SBATCH --exclusive

module purge
module restore swift_module

mpirun -np 4 ./swift_mpi -a -s -G -t 14 parameters_impact.yml 

```

#### **<font color='green'>Multi-node job with <font color='red'>partial</font> cpus</font>** 

```
#!/bin/bash -l

#SBATCH -J swift_impact_2node32cpu
#SBATCH --cpus-per-task=8
#SBATCH --tasks-per-node=2
#SBATCH --nodes 2

module purge
module restore swift_module

mpirun -np 4 ./swift_mpi -a -s -G -t 8 parameters_impact.yml 
```

### <a name="S15"></a>**Command line options**

* Above script only contains the very basic SLURM skeleton, you can always add other flags to limit the time, memory, partitions, emails, etc. 
* You can check the command line options list by using `./swift -h`. 
* Normally, for the planetary impact simulation, `-s` (stands for running with hydrodynamics) and `-G` (stands for running with self-gravity) are two must have options. 
* `-t <int>` is used to pass on the the number of task threads to use on each MPI rank. If you are running on one node, `-t` can be as large as the total number of threads or cpus the node has. See above "Single-node job with all cpus" script for example. If running with MPI, `-t` can be as large as the number of cpus per chip or socket has. See above "Multi-node job with all cpus" script for example. 
* Typically, HPC clusters now use two chips per node, so in the script, make sure you add `#SBATCH --tasks-per-node=2`, this will let SWIFT run with one MPI rank per NUMA region.
* The `mpirun` comamnd should follow `mpirun -np <number of chips in total> ./swift_mpi -s -G -t <number of cpus(cores) per chip>`. Take BlueCrystal 4 as example: each node has two chips and each chips has 14 cpus, so if using 4 nodes in total ( 8 chips in total then ), it would be like `mpirun -np 8 ./swift_mpi -s -G -t 14`
* Threads pinned. You can do this by passing the `-a` flag to the SWIFT binary. This ensures that processes stay on the same core that spawned them, ensuring that cache is accessed more efficiently.
* Restart. If restart is enabled in the parameters file, you can just restart the job from the most recently generated restart file. Just pass `-r` flag to the SWIFT binary, i.e.,`./swift -r -s -G -t 24`
* Verbose output. Verbose output can be enabled with `-v 1` flag. If runing muti-node job, `-v 2` can be used to output from all ranks. Be careful with the verbose output, logfile can be quit large and will take up all your home directories quota. Better to generate the logfile in the scrath space then.
* Steps. If you want to use number of steps instead simulation time to control the job, pass `-n <int>` to the swift binary. Job will terminate after certain steps. This can be useful when combining with restart to debug.

[Back to Contents](#top)

## <a name="S2"></a>2. Parameter file

```python
# Define the system of units to use internally.
InternalUnitSystem:
    UnitMass_in_cgs:        5.97240e27         # Set the internal mass unit to Earth mass = 5.97240e27 g
    UnitLength_in_cgs:      6.371e8            # Set the internal length unit to Earth radius = 6.371e8 cm
    UnitVelocity_in_cgs:    6.371e8            # Set time in seconds (SWIFT set internal velocity insead of time)
    UnitCurrent_in_cgs:     1.0                # Amperes
    UnitTemp_in_cgs:        1.0                # Kelvin

# Parameters related to the initial conditions
InitialConditions:
    file_name:  ./swift_impact.hdf5      # The initial conditions file to read
    periodic:   0                        # Are we running with periodic ICs?

# Parameters governing the time integration
TimeIntegration:
    time_begin:     0                   # The starting time of the simulation (in internal units).
    time_end:       72000               # The end time of the simulation (in internal units).
    dt_min:         0.0001              # The minimal time-step size of the simulation (in internal units).
    dt_max:         1000                # The maximal time-step size of the simulation (in internal units).

# Parameters governing the snapshots
Snapshots:
    basename:           snapshot        # Common part of the name of output files
    time_first:         0               # Time of the first output (in internal units)
    delta_time:         100             # Time difference between consecutive outputs (in internal units)
    subdir:             ./output        # Output directory, will automatically generate one if not find at the start of the simulation   

# Parameters governing the conserved quantities statistics
Statistics:
    time_first: 0                       # Time of the first output (in internal units)
    delta_time: 1000                    # Time between statistics output

# Parameters controlling restarts
Restarts:
    enable:         1                   # Whether to enable dumping restarts at fixed intervals.
    save:           1                   # Whether to save copies of the previous set of restart files (named .prev)
    onexit:         1                   # Whether to dump restarts on exit (*needs enable*)
    subdir:         ./RESTART           # Name of subdirectory for restart files.
    basename:       Rfile               # Prefix used in naming restart files.
    delta_hours:    0.5                 # Decimal hours between dumps of restart files. NOTE!!! Unit change to hours(h) here.
    
SPH:
    resolution_eta:     1.2348          # Target smoothing length in units of the mean inter-particle separation (1.2348 == 48Ngbs with the cubic spline kernel).
    delta_neighbours:   0.1             # The tolerance for the targetted number of neighbours.
    CFL_condition:      0.2             # Courant-Friedrich-Levy condition for time integration.
    h_max:              0.08            # Maximal allowed smoothing length (in internal units). 
    viscosity_alpha:    1.5             # Override for the initial value of the artificial viscosity.

# Parameters for the self-gravity scheme
Gravity:
    eta:                            0.025       # Constant dimensionless multiplier for time integration.
    MAC:                            adaptive    # Choice of mulitpole acceptance criterion: 'adaptive' OR 'geometric'.
    epsilon_fmm:                    0.001       # Tolerance parameter for the adaptive multipole acceptance criterion.
    theta_cr:                       0.5         # Opening angle for the purely gemoetric criterion.
    max_physical_baryon_softening:  0.05        # Physical softening length (in internal units).
    use_tree_below_softening:       0           # Whether or not to use the approximate gravity from the FMM tree below the softening scale

# Parameters governing domain decomposition
DomainDecomposition:
    trigger:        0.1                 # Fractional (<1) CPU time difference between MPI ranks required to trigger a new decomposition, or number of steps (>1)                                             # between decompositions 
    adaptive:         0                 # Use adaptive repartition when ParMETIS is available, otherwise simple refinement.
    
# Parameters that control how the cell tree is configured and defines some values for the related tasks. Tuning parameters, both for speed and memory use.
Scheduler:
    max_top_level_cells:    16         # Maximal number of top-level cells in any dimension. The number of top-level cells will be the cube of this. 
    cell_split_size:        400         # Maximal number of particles per cell (400 is the default value).
    tasks_per_cell:         2.5         # The average number of tasks per cell. If not large enough the simulation will fail (means guess...). 
    links_per_tasks:        20          # number of links per task
    mpi_message_limit:      4096        # Defines the size (in bytes) below which MPI communication will be sent using non-buffered calls.
    nr_queues:              28          # Defines the number of task queues used. These are normally set to one per thread and should be at least that number.

# Parameters related to the equation of state
EoS:
    # Select which planetary EoS material(s) to enable for use.
    # Have a look at http://swift.dur.ac.uk/docs/Planetary/equations_of_state.html if you want to use other kind of EoS
    
    planetary_use_ANEOS_forsterite:       1     #  material id 400
    planetary_use_ANEOS_iron:             1     #  material ID 401
    planetary_ANEOS_forsterite_table_file:      ./ANEOS_forsterite_S19.txt
    planetary_ANEOS_iron_table_file:            ./ANEOS_iron_S20.txt  

```

---

You will find a parameter file called `parameters_impact.yml` in the repository alongside with this notebook. Normally, you will need to change the following parameters:
* `file_name`  what's the name of the initial condition hdf5 file
* `time_end`   simulation time in second
* `delta_time` Time difference in second between consecutive outputs
* `subdir`     name of the output directory
* `h_max` Particles with h larger than this h_max limit will be capped and are effectively treated as just balistic objects. Currently, we use 5%~10% of the radius of the largest planet in the simulation as the value for h_max. h_max will greatly affect the speed of SWIFT in terms of planetary impact. Current experience is that do not use `h_max` larger than 1/6 of the boxsize, otherwise a potential bug in SWIFT would occur and the simulation result is bad. If you turn verbose output on, you might notice the reported `h_max` is different than what you set initially in the parameter file, this is because the `h_max` reported here is the maximum smoothing length among all the gas particles at this step. It's different because the largest smoothing length at current step is still less than the limit you set in parameter file. `h_max` and `cell_min` are all in internal unit. Some of the discussion about h_max can be found [here](https://github.com/SWIFTSIM/swiftsim/issues/22).
* `max_top_level_cells` Normally set to 16 and only make it larger if the gravity solver gets very slow.
* `tasks_per_cell` Need a larger value e.g. 3.0, if using high resolution but not enough number of cpus. 
* `nr_queues`  Set to one per thread and should be at least that number.

[Back to Contents](#top)

## <a name="S3"></a>3. Generating IC file

All the initial condition file from my tests are generated using the WOMA module. You can find a jupyter notebook tutorial from their [github](https://github.com/srbonilla/WoMa). You need to install WOMA if you want to use it to generate your IC file, otherwise, you can use the swiftsimio python package to generate the IC file directly. Here is the swiftsimio [github](https://github.com/SWIFTSIM/swiftsimio) and [documentation](https://swiftsimio.readthedocs.io/en/latest/index.html). swiftsimio is more often used to load the swift hdf5 data, check this [link](https://swiftsimio.readthedocs.io/en/latest/loading_data/index.html) to know how to load data.

NOTE!!! WOMA use <font color='red'> mks unit system </font>while Gadget2 use cgs, so please always be careful with the unit.

Below is a very simple example of how to use WOMA to generate a two-layer planetsimal.  

In [22]:
import woma
import swiftsimio as sw
import numpy as np
import unyt
import h5py
R_earth = 6.371e6   # m
M_earth = 5.9724e24  # kg m^-3 
G = 6.67408e-11  # m^3 kg^-1 s^-2

In [4]:
planet = woma.Planet(
    name            = "target_planet",
    A1_mat_layer    = ["ANEOS_iron", "ANEOS_forsterite"], # check the name of different material and EoS at: https://github.com/srbonilla/WoMa/blob/master/woma/misc/glob_vars.py
    A1_T_rho_type   = ["entropy=1500", "entropy=3500"],   # Here I use fixed entropy for each layer, other option: "adiabatic","power=<float>"
    P_s             = 1e4,                                # Surface pressure
    T_s             = 1500,                               # Surface temperature
    A1_M_layer      = [0.3*M_earth, 0.7*M_earth],         # Masses of each layer
)
planet.gen_prof_L2_find_R_R1_given_M1_M2(R_min=0.95*R_earth, R_max=1.05 * R_earth) # This will only generate the radius profile
planet_particles = woma.ParticlePlanet(planet, 5e4, verbosity=0) # This will place 5e4 particles based on the profile generated at last step.
                                                                 # Note the final total number of particles can not be controled and this is 
                                                                 # caused by their tuning method of WOMA.
# Save to the hdf5 file
filename='/data/storage/planet.hdf5'
planet_particles.save(
    filename=filename,
    boxsize=10*R_earth,                                         # set the boxsize
    file_to_SI=woma.Conversions(M_earth, R_earth, 1),           # This will use the earth radius and mass as the storage unit
    #file_to_SI=utils.SI_to_cgs                                 # You can convert to cgs unit using this command instead
    do_entropies=True)                                          # If you want to use entropy forcing, make sure "do_entropies" is set to True.

Trying to build a planet with R=R_min... Success
Trying to build a planet with R=R_max... Success
Iter 9(40): R=1.0244R_E R1=0.51271R_E: tol=0.0003(0.001)  
Tweaking M to avoid density peaks at the center of the planet... Done
Planet "target_planet": 
    M            = 5.9657e+24  kg  = 0.99888  M_earth
    R            = 6.5265e+06  m  = 1.0244  R_earth
    mat          = ["ANEOS_iron", "ANEOS_forsterite"] 
    mat_id       = [401, 400] 
    T_rho_type   = ["entropy=1500", "entropy=3500"] 
    R_layer      = [0.51169, 1.0244]  R_earth
    M_layer      = [0.29899, 0.69989]  M_earth
    M_frac_layer = [0.29933, 0.70067]  M_tot
    idx_layer    = [499, 999] 
    P_s          = 10000  Pa
    T_s          = 1500  K
    rho_s        = 3110.5  kg m^-3
    P_1          = 1.4014e+11  Pa
    T_1          = 2738.2  K
    rho_1        = 11156  kg m^-3
    P_0          = 3.8559e+11  Pa
    T_0          = 3588.6  K
    rho_0        = 13797  kg m^-3
    I_MR2        = 0.32451  M_tot*R_tot^2


Here is the temperature and density power law relationship if you choose to use `A1_T_rho_type=["power=alpha"] `$T\propto\rho^\alpha$ ($\alpha=0$ for isothermal).
You could then use SWIFT to equilibrate the planetsimals.

After artifical cooling and relax check. You could use the following code to load the SWIFT snapshot and convert the different instance to the mks unit used by WOMA. For detailed document, check swiftsimio [guid](https://swiftsimio.readthedocs.io/en/latest/index.html).

In [17]:
def load_to_woma(snapshot):
    # Load
    data    = sw.load(snapshot)
    
    data.gas.coordinates.convert_to_mks()
    data.gas.velocities.convert_to_mks()
    data.gas.smoothing_lengths.convert_to_mks()
    data.gas.masses.convert_to_mks()
    data.gas.densities.convert_to_mks()
    data.gas.pressures.convert_to_mks()
    data.gas.internal_energies.convert_to_mks()
    box_mid = 0.5*data.metadata.boxsize[0].to(unyt.m)

    pos     = np.array(data.gas.coordinates - box_mid)
    vel     = np.array(data.gas.velocities)
    h       = np.array(data.gas.smoothing_lengths)
    m       = np.array(data.gas.masses)
    rho     = np.array(data.gas.densities)
    p       = np.array(data.gas.pressures)
    u       = np.array(data.gas.internal_energies)
    matid   = np.array(data.gas.material_ids)
    
    pos_centerM = np.sum(pos * m[:,np.newaxis], axis=0) / np.sum(m)
    vel_centerM = np.sum(vel * m[:,np.newaxis], axis=0) / np.sum(m)
    
    pos -= pos_centerM
    vel -= vel_centerM
    
    xy = np.hypot(pos[:,0],pos[:,1])
    r  = np.hypot(xy,pos[:,2])
    r  = np.sort(r)
    R  = np.mean(r[-100:])
    
    return pos, vel, h, m, rho, p, u, matid, R

You could use the following code to load and generate a complete impact simulation initial condition file.

In [18]:
loc_tar='/data/trappist1/Jingyao/swift/planetary/pb1data/output_swift_05715_65e5_EF_10h/snapshot_0360.hdf5'
loc_imp='/data/trappist1/Jingyao/swift/planetary/pb1data/output_swift_0468_65e5_EF_10h/snapshot_0360.hdf5'

pos_tar, vel_tar, h_tar,m_tar, rho_tar, p_tar, u_tar, matid_tar,pid_tar, R_tar = load_to_woma(loc_tar)
pos_imp, vel_imp, h_imp,m_imp, rho_imp, p_imp, u_imp, matid_imp,pid_imp, R_imp = load_to_woma(loc_imp)

In [23]:
M_t = np.sum(m_tar)
M_i = np.sum(m_imp)
R_t = R_tar
R_i = R_imp

# Mutual escape speed
v_esc = np.sqrt(2 * G * (M_t + M_i) / (R_t + R_i))

# Initial position and velocity of the target
A1_pos_t = np.array([0., 0., 0.])
A1_vel_t = np.array([0., 0., 0.])

A1_pos_i, A1_vel_i = woma.impact_pos_vel_b_v_c_t(
    b       = 0.3,
    v_c     = 3.0 * v_esc, 
    t       = 3600, 
    R_t     = R_t, 
    R_i     = R_i, 
    M_t     = M_t, 
    M_i     = M_i,
)

A1_pos_com = (M_t * A1_pos_t + M_i * A1_pos_i) / (M_t + M_i)
A1_pos_t -= A1_pos_com
A1_pos_i -= A1_pos_com

# Centre of momentum
A1_vel_com = (M_t * A1_vel_t + M_i * A1_vel_i) / (M_t + M_i)
A1_vel_t -= A1_vel_com
A1_vel_i -= A1_vel_com

pos_tar += A1_pos_t
vel_tar[:] += A1_vel_t

pos_imp += A1_pos_i
vel_imp[:] += A1_vel_i


with h5py.File("/loc/of/your/dir/foo.hdf5", "w") as f:
    woma.save_particle_data(
        f,
        np.append(pos_tar, pos_imp, axis=0),
        np.append(vel_tar, vel_imp, axis=0),
        np.append(m_tar, m_imp),
        np.append(h_tar, h_imp),
        np.append(rho_tar, rho_imp),
        np.append(p_tar, p_imp),
        np.append(u_tar, u_imp),
        np.append(matid_tar, matid_imp),
        boxsize=100 * R_earth,
        file_to_SI=woma.Conversions(M_earth, R_earth, 1),
    )

Number of particles in target 645646

num_particle = 1290680
boxsize      = 1e+02
mat_id       = 400 401 

Unit mass    = 5.97240e+27 g
Unit length  = 6.37100e+08 cm
Unit time    = 1.00000e+00 s

Min, max values (file units):
  pos = [45.589, 55.151,    48.562, 51.518,    49.143, 50.858]
  vel = [-0.0012135, 0.00099127,    -0.00018171, 0.00021851,    -2.4332e-05, 2.4174e-05]
  m = 7.2158e-07, 8.8378e-07
  rho = 0.1355, 0.53601
  P = 6.1353e-10, 2.5155e-07
  u = 2.8456e-08, 1.693e-07
  h = 0.01383, 0.023054

Saved "o/swift/planetary/ics/impact_05715_0468_65e5_9d7_055_100box.hdf5"


[Back to Contents](#top)

## <a name="S4"></a>4. Read and Analysis

In [1]:
woma.load_eos_tables() # load the eos table for the calculation of entropy at later stage

def sw_load(file_loc, ax_lim=1.3, Entropy=True, idealgas= False, center=True):
    # Load
    #woma.load_eos_tables()
    
   
    data    = sw.load(file_loc)
    box_mid = 0.5*data.metadata.boxsize[0]
    pos     = data.gas.coordinates - box_mid
    pos =np.array(pos)
    data.gas.densities.convert_to_cgs()
    rho_cgs = np.array(data.gas.densities)
    data.gas.densities.convert_to_mks()
    rho_mks = np.array(data.gas.densities)
    data.gas.pressures.convert_to_cgs()
    p       = np.array(data.gas.pressures)
    data.gas.internal_energies.convert_to_mks()
    u       = np.array(data.gas.internal_energies)
    matid   = data.gas.material_ids
    data.gas.pressures.convert_to_mks()
    p_mks   = np.array(data.gas.pressures)
    matid   = np.array(matid)
    
    colour = np.empty(len(matid), dtype=object)
    for id_c, c in Di_id_colour.items():
        colour[matid == id_c] = c

    
    entropy = np.zeros_like(p)
    T       = np.zeros_like(p)
    
    if center:
        #data.gas.masses.convert_to_mks()
        m       = np.array(data.gas.masses)
        pos_centerM = np.sum(pos * m[:,np.newaxis], axis=0) / np.sum(m)
        #vel_centerM = np.sum(vel * m[:,np.newaxis], axis=0) / np.sum(m)
    
        pos -= pos_centerM
        #vel -= vel_centerM
    
    if Entropy:
        sel = np.logical_and(matid!=200, matid!=0)  
        entropy[sel] = woma.eos.eos.A1_s_u_rho(u[sel],rho_mks[sel],matid[sel])*1e4
        #T       = woma.eos.eos.A1_T_u_rho(u,rho_mks,matid)
        
    
    T       = woma.eos.eos.A1_T_u_rho(u,rho_mks,matid)
    
    XY = np.hypot(pos[:,0],pos[:,1]) 
    R  = np.hypot(XY,pos[:,2])
           
    print('Read %d particles!'%(len(R)))
    
    return T, entropy, u, rho_cgs, p_mks, R, colour, matid



NameError: name 'woma' is not defined

Below code could be used to load and plot data from SWIFT simulation.

In [ ]:
outtime   =500
txtsize   =15
font_size = 20
idoff     = 200000000
bodyoff   = 100000000

params = {
    "axes.labelsize"  : font_size,
    "font.size"       : font_size,
    "xtick.labelsize" : font_size,
    "ytick.labelsize" : font_size,
    "font.family"     : "serif",
}
matplotlib.rcParams.update(params)

# Material IDs ( = type_id * type_factor + unit_id )
type_factor  = 100
type_Til     = 4
type_HHe     = 2
type_SESAME  = 3
type_idg     = 0
id_body      = 200000000
# Name and ID
Di_mat_id = {
    "ANEOS_iron"     : type_Til * type_factor + 1 ,
    "ANEOS_iron_2"   : type_Til * type_factor + 1 + id_body,
    "ANEOS_alloy"    : type_Til * type_factor + 2 ,
    "ANEOS_alloy_2"  : type_Til * type_factor + 2 + id_body,
    "ANEOS_granite"  : type_Til * type_factor ,
    "ANEOS_granite_2": type_Til * type_factor + id_body,
    "HM80_HHE"       : type_HHe * type_factor,
    "SS08_water"     : type_SESAME * type_factor + 3,
    "AQUA"           : type_SESAME * type_factor + 4,
    "CMS19_HHe"      : type_SESAME * type_factor + 7,
    "idg_HHe"        : type_idg * type_factor,
}
# Colour
Di_mat_colour = {
    "ANEOS_iron"     : "tomato",
    "ANEOS_alloy"    : "tomato",
    "ANEOS_granite"  : "mediumseagreen",
    "ANEOS_iron_2"   : "sandybrown",
    "ANEOS_alloy_2"  : "sandybrown",
    "ANEOS_granite_2": "pink",
    "SS08_water"     : "skyblue",
    "AQUA"           : "skyblue",
    "HM80_HHE"       : "lavenderblush",
    "CMS19_HHe"      : "lavenderblush",
    "idg_HHe"        : "lavenderblush",
}
# marker size
Di_mat_size = {
    "ANEOS_iron"     : 1,
    "ANEOS_alloy"    : 1,
    "ANEOS_granite"  : 1,
    "ANEOS_alloy_2"  : 1,
    "ANEOS_iron_2"   : 1,
    "ANEOS_granite_2": 1,
    "HM80_HHE"       : 1,
    "SS08_water"     : 1,
    "AQUA"           : 1,
    "CMS19_HHe"      : 1,
    "idg_HHe"        : 1,
}

Di_id_colour = {Di_mat_id[mat]: colour for mat, colour in Di_mat_colour.items()}

Di_id_size = {Di_mat_id[mat]: size*3.5 for mat, size in Di_mat_size.items()}

# density limits
rhomin=5e-5
rhomax=15.

# entropy limits
cmin=1.5
cmax=12.

# number of cells for grid
Ng=601j
Ngz=21j

# region to grid/plot
xmax=3
xmin=-xmax
ymin=-xmax
ymax=xmax
zmin=-3
zmax=3

zcut=6.

def sw_plot(loc, snapshot_id, npt=1e9, ax_lim=3.0,offcenter=0, text=False, midplane=False,rhocontour=False, quarter=False, belowZ=True, figsz=10):
    """ Select and load the particles to plot. """
    # Snapshot to load
    snapshot = "snapshot_%04d.hdf5" % snapshot_id

    # Only load data with the axis limits and below z=0
    mask = sw.mask(loc+snapshot)
    box_mid = 0.5 * mask.metadata.boxsize[0].to(unyt.Rearth)
    
    # Load
    data = sw.load(loc+snapshot)
    pos = data.gas.coordinates.to(unyt.Rearth) - box_mid
    id = data.gas.particle_ids
    mat_id = data.gas.material_ids.value
    m  = data.gas.masses

    if rhocontour:
        s=Snapshot()
        zi=np.linspace(zmin,zmax,int(Ngz.imag))
        X,Y,Z = np.mgrid[xmin:xmax:(Ng),ymin:ymax:(Ng),zmin:zmax:(Ngz)]
        XX,YY = np.mgrid[xmin:xmax:(Ng),ymin:ymax:(Ng)]

        cmap=plt.get_cmap('plasma').copy()
        cmap.set_under('w')

        data.gas.velocities.convert_to_cgs()
        vel = np.array(data.gas.velocities)

        data.gas.densities.convert_to_cgs()
        rho_cgs=np.array(data.gas.densities)
        
        s.pot=data.gas.potentials
        s.vel=np.array(vel)
        s.vx=np.array(vel[:,0])
        s.vy=np.array(vel[:,1])
        s.vz=np.array(vel[:,2])
        s.x=np.array(pos[:,0])
        s.y=np.array(pos[:,1])
        s.z=np.array(pos[:,2])
        s.rho=np.array(rho_cgs)
        
        modz = np.abs(s.z)
    
        vcut=2*s.vel.max()
        
        rhoi = scipy.interpolate.griddata((s.x[(s.vx<vcut)*(modz<zcut)],s.y[(s.vx<vcut)*(modz<zcut)],s.z[(s.vx<vcut)*(modz<zcut)]),\
                                          s.rho[(s.vx<vcut)*(modz<zcut)],(X,Y,Z),method='linear',fill_value=1.e-18)
        
        coz = (s.z[s.pot==s.pot.min()])[0] #s.z[modz<zcut]
        nn=(npy.nonzero(zi==(zi[zi<=coz])[-1])[0])[0]

        RHO_sh=rhoi[:,:,nn].T
        step = 0.8
        m = np.amax(RHO_sh)
        levels = np.arange(1.0, m, step)

    pos_centerM = np.sum(pos * m[:,np.newaxis], axis=0) / np.sum(m)
    pos -= pos_centerM
    
    if midplane:
        sel    = np.where(np.logical_and(pos[:, 2] < 0.1, pos[:, 2] > -0.1))[0]
        mat_id = mat_id[sel]
        m      = m[sel]
        id     = id[sel]
        pos    = pos[sel]
   
    
    #Restrict to z < 0
    elif belowZ:
        sel    = np.where(np.logical_and(pos[:, 2] < 0.1, pos[:, 2] > -6))[0]
        mat_id = mat_id[sel]
        id     = id[sel]
        m      = m[sel]
        pos    = pos[sel]

    # Sort in z order so higher particles are plotted on top
    sort   = np.argsort(pos[:, 2])
    mat_id = mat_id[sort]
    id     = id[sort]
    m      = m[sort]
    pos    = pos[sort]
    
    
    if npt>0:
        mat_id[npt<=id]+=id_body
    else:
        mat_id[np.logical_and(id<=2*idoff,id>idoff+bodyoff)] += id_body
        mat_id[np.logical_and(id<=idoff,id>bodyoff)] += id_body
    
    impactor_core   = np.logical_and(mat_id==Di_mat_id["ANEOS_iron_2"], pos[:,1]<-2)
    target_core     = mat_id==Di_mat_id["ANEOS_iron"]
    impactor_mantle = mat_id==Di_mat_id["ANEOS_granite_2"]
    target_mantle   = mat_id==Di_mat_id["ANEOS_granite"]
    
    #Plot the particles, coloured by their material.
    fig = plt.figure(num=1, clear=True,figsize=(figsz, figsz))
    ax = plt.gca()
    ax.set_aspect("equal")

    colour = np.empty(len(pos), dtype=object)
    for id_c, c in Di_id_colour.items():
        colour[mat_id == id_c] = c

    size = np.empty(len(pos), dtype=float)
    for id_s, s in Di_id_size.items():
        size[mat_id == id_s] = s
    print(np.unique(mat_id))    
    print(set(colour))
    #return pos, size, colour, impactor_core
    ax.scatter(
        pos[:, 0],
        pos[:, 1],
        s=1*size,
        c=colour,
        edgecolors="none",
        marker=".",
        alpha=1,
    )
   
   
#     ax.scatter(pos[target_mantle,0],pos[target_mantle,1],s=1,c='mediumseagreen',edgecolors="none")
    ax.scatter(pos[impactor_core,0],pos[impactor_core,1],s=1,c='sandybrown',edgecolors="none")
#     ax.scatter(pos[target_core,0],pos[target_core,1],s=1,c='tomato',edgecolors="none")
    if rhocontour:
        con = ax.contour(XX,YY,RHO_sh, levels, origin='lower',extent=[-ax_lim,ax_lim,-ax_lim,ax_lim],alphas=0,cmap='YlGn')
    
    #ax.set_title('0.897 $M_\oplus$ Protoearth with 1.2% $M_{total}$ atmosphere ')
    ax.set_xlim(-(ax_lim+offcenter), ax_lim-offcenter)
    ax.set_yticks(ax.get_xticks())
    ax.set_ylim(-ax_lim, ax_lim)
    ax.set_xlabel(r"x Position ($R_\oplus$)")
    ax.set_ylabel(r"y Position ($R_\oplus$)")
    
    ax.yaxis.label.set_color('w')
    ax.xaxis.label.set_color('w')
    ax.tick_params(axis='x', colors='w',labelsize=14)
    ax.tick_params(axis='y', colors='w',labelsize=14)
    
    if quarter:
        ax.set_xlim(0, ax_lim+0.2)
        ax.set_ylim(0, ax_lim+0.2)
    
        
    if text:
        ax.text(0.97,0.97,r'v = 9.17km/s, b = 0.7            {0:6.2f} h'.format(snapshot_id*outtime/3600),
                     transform=ax.transAxes,ha='right',va='top',fontsize=txtsize,color='w')

    plt.tight_layout()
    plt.show()
    # save = loc + "plot/snaplot_%04d.png" % snapshot_id
    # plt.savefig(save)
    plt.cla()
    plt.clf() 
    plt.close()

[Back to Contents](#top)